In [46]:
# This file is used for demo purposes only

In [47]:
import spacy

import re
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import nltk
import json

from spacy.language import Language
from typing import Dict, List, Set, Tuple
from collections import defaultdict, namedtuple
import copy
from statistics import mean 
import io
from PIL import Image, ImageDraw, ImageFont
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from google.protobuf.json_format import MessageToJson, Parse
from google.protobuf import json_format

import os
import sys
sys.path.append("../")
from models.dish_segmenter import Dish
from models.bounding_box import FeatureType, Point, BoundingBox, DSU
from models.word_unit import WordUnit
from utils.cv_preprocess import *
from utils.file_utils import *
from utils.nlp_preprocess import *
from IPython.display import display


In [48]:

def process_menu(filein):
    base_name = os.path.basename(filein)
    file_name_without_extension = os.path.splitext(base_name)[0]

    raw_ocr_directory = '../dataset/menu_photo_ocr_raw/'
    raw_ocr_filename = file_name_without_extension + "_raw_annotation.json"
    raw_ocr_path = os.path.join(raw_ocr_directory, raw_ocr_filename)

    document = load_json(raw_ocr_path)
    image = Image.open(filein)
    
    

    bounds = process_bounds_in_words(document)
    
    # draw_boxes_2_points(image, bounds, (255, 0, 0))
    # plt.figure(figsize=(10, 10))
    # display(image)

    price_bounds = extract_price_bounds(bounds)
    filtered_bounds, chinese_bbox, english_bbox = filter_and_classify_bounds(bounds)

    # locator_bounds = [price_bounds, chinese_bbox]
    # locator_bounds = {"price_bounds": price_bounds}
    locator_bounds = {"chinese_bbox": chinese_bbox, "price_bounds": price_bounds}
    # locator_bounds = {"chinese_bbox": chinese_bbox}

    overlap_threshold_list = [0.3, 0.4, 0.5]

    container_width, container_height = image.size
    
    # loop through all possible extend directions
    extend_directions_pair = {"price_bounds":[[ExtendDirection.BOTTOM, ExtendDirection.LEFT], [ExtendDirection.BOTTOM, ExtendDirection.RIGHT]],
                                "chinese_bbox": [[ExtendDirection.BOTTOM, ExtendDirection.LEFT], [ExtendDirection.BOTTOM, ExtendDirection.RIGHT], [ExtendDirection.TOP, ExtendDirection.LEFT], [ExtendDirection.TOP, ExtendDirection.RIGHT]]
                              }
    # extend_directions_pair = [[ExtendDirection.BOTTOM, ExtendDirection.RIGHT]]
    
    # find the extend direction that have the highest semantic correlation in chinese and english
    max_correlation = 0
    max_correlation_pair = None
    max_locator = None
    max_overlap = 0
    max_avg_correlation = 0

    max_grouped_list, max_grouped_box = None, None  
    for overlap_threshold in overlap_threshold_list:
        for locator_name, locator_bound in locator_bounds.items():

            for extend_directions in extend_directions_pair[locator_name]:
                # print()
                # print("Processing", extend_directions)
                extended_boxes = extend_bounding_boxes(locator_bound, container_width, container_height, extend_directions=extend_directions)
                sorted_bounding_boxes = sorted(filtered_bounds, key=lambda bbox: (bbox.y_min, bbox.x_min))
                grouped_list, grouped_box = group_extended_boxes(extended_boxes, sorted_bounding_boxes, overlap_threshold=overlap_threshold)
                
                total_correlation = 0
                correlation_count = 0
                for string_list in grouped_list:
                    # flatten the list of strings by joining them
                    string_list = [" ".join(string) for string in string_list if string != ""]
                    
                    chinese_text, english_text = split_chinese_english(string_list)
                    chinese_text = "".join(chinese_text)
                    english_text = " ".join(english_text)
                    
                    correlation = calculate_semantic_correlation(chinese_text, english_text)
                    # print(chinese_text, english_text, correlation)
                    total_correlation += correlation
                    correlation_count += 1
                
                
                # Calculate average correlation if there are valid correlations
                
                avg_correlation = total_correlation / correlation_count if correlation_count > 0 else 0
                # print("Avg correlation", total_correlation)
                if total_correlation > max_avg_correlation:
                    max_avg_correlation = total_correlation
                    max_correlation_pair = extend_directions
                    max_grouped_list = grouped_list
                    max_grouped_box = grouped_box
                    max_locator = locator_bound
                    max_overlap = overlap_threshold
    
    # print("Max correlation", max_avg_correlation)
    # print("Max correlation pair", max_correlation_pair)
    # print("Max overlap", max_overlap)
    
    draw_boxes_2_points(image, max_grouped_box, (0, 255, 0))
    plt.figure(figsize=(10, 10))
    display(image)
    
    dish_instance_list = []
    
    for string_list in max_grouped_list:
        # print(string_list)
        dish = segment_dish_text_list(string_list)
        # print(dish)
        dish_instance_list.append(dish)
        
    return max_grouped_list, max_grouped_box

# Usage example
# filein = '../dataset/menu_photo/segment_by_chinese/5._Ma_La_Sichuan_0_page_8.png'
# filein = '../dataset/menu_photo/segment_by_chinese/menu-final_page_18.png'

# filein = '../dataset/menu_photo/segment_by_chinese/Rendezvous_sit-in_08-23_Draft_page_3.png'
# filein = '../dataset/menu_photo/segment_by_chinese/Home Dim Sum Daytime Menu _page_0.png'
# filein = '../dataset/menu_photo/segment_by_chinese/Home Dim Sum Daytime Menu _page_1.png'
filein = '../dataset/menu_photo/segment_by_chinese/3._Chop_Chop_2_segment_0.jpg'





process_menu(filein)


([[['海鮮', '炒麵', '河'],
   ['Stir', 'Fried', 'Seafood', 'Noodles'],
   ['Soft', 'egg', 'noodles', 'Dry', 'kcal'],
   ['Crispy', 'egg', 'noodles', 'kcal'],
   ['Ho', 'fun', 'noodles', 'kcal']],
  [['乾炒', '河'],
   ['Stir', 'Fried', 'Ho', 'Fun', 'Noodles', 'Dry'],
   ['Beef', 'kcal'],
   ['Mixed', 'Vegetables', 'VG', 'kcal']],
  [['豉', '椒', '炒'],
   ['Stir', 'Fried', 'in', 'Black', 'Bean', 'sauce'],
   ['Crispy', 'egg', 'noodles', 'kcal'],
   ['Ho', 'fun', 'noodles', 'kcal'],
   ['Steamed', 'rice', 'kcal'],
   ['Chicken', 'kcal'],
   ['Beef', 'kcal'],
   ['Prawns', 'kcal'],
   ['Seafood', 'kcal'],
   ['Tofu', 'VG', 'kcal']],
  [['星洲', '炒'],
   ['Singapore', 'Fried'],
   ['Pork', 'and', 'Prawn', 'vermicelli', 'noodles', 'P', 'kcal'],
   ['Pork', 'and', 'Prawn', 'rice', 'P', 'kcal']]],
  <models.bounding_box.BoundingBox at 0x175e9fc10>])

<Figure size 1000x1000 with 0 Axes>